In [277]:
# 导入库
import paho.mqtt.client as ph_mqtt_clt
import json
import threading
import base64
import requests
import time
from enum import Enum
from collections import deque


In [278]:
class Mqtt_Clt():
    """
    用于表征Mqtt客户端的类
    """
    def __init__(self, ip_broker, port_broker, topic_sub, topic_pub, time_out_secs):
        """
        parameters:ip_broker:broker的IP地址
                   port_broker:连接服务的端口
                   topic_sub:订阅话题/订阅名称
                   topic_pub:发布话题/发布名称
                   time_out_secs:连接超时的时间
        """
        self.mqtt_clt = ph_mqtt_clt.Client()    # 构造Mqtt客户端
        self.mqtt_clt.on_message = self.on_message  # 设置Mqtt客户端回调on_message为本类成员方法on_message
        self.topic_sub = topic_sub  # 设置订阅话题
        self.topic_pub = topic_pub  # 设置发布话题
        self.msg = {}   # 初始化接收消息的字典类成员变量msg为空({})
        self.mqtt_clt.connect(ip_broker, port_broker, time_out_secs)    # 连接Mqtt Broker
        self.mqtt_clt.subscribe(self.topic_sub, qos=2)  # 订阅相关话题
        self.mqtt_clt.loop_start()  # 开启接收循环 开启独立的线程来循环是否有消息通讯

    def __del__(self):
        self.mqtt_clt.loop_forever()    # 结束接收循环
        self.mqtt_clt.disconnect()  # 断开连接


    def on_message(self, client, userdata, message):
        """
        parameters:client:回调返回的客户端实例
                   userdata:Client()或user_data_set()中设置的私有用户数据
                   msg:MQTTMessage实例,包含topic,payload,qos,retain
        functions:接收消息的回调,提取消息中的相关内容
        """
        # self.msg = json.loads(message.payload.decode())
        # 这里用其他线程来解放on_message独占线程的问题
        threading.Thread(target=self.handle_on_message,args=(message,)).start()
        # print("%%%%%%%%%%%%")
        # print("msg_json:", self.msg)
    
    def handle_on_message(self,message):
        self.msg = json.loads(message.payload.decode())

    def send_json_msg(self, msg):
        """
        parameters:msg:json消息
        returns:无
        functions:在指定的话题上发布消息
        """
        # retain参数:一个 topic 只能存在一条 Retained 消息，发布新的 Retained 消息将会覆盖旧消息
        # 只有新的订阅者才能够收到 Retained 消息
        # MQTT客户端 好像是单线程的，无法同时处理订阅和发布 这里要用其他线程来实现
        threading.Thread(target=self.mqtt_clt.publish(self.topic_pub, payload="{}".format(msg))).start()
        # info = self.mqtt_clt.publish(self.topic_pub, payload="{}".format(msg))
        # print("published:True",end=" ") if info.is_published else print("publish:False",end=" ")
        print(msg)
        
    def control_device(self, str_key, str_value):
        """
        parameters:str_key:键的字符串
                   str_value:值的字符串
        functions:控制设备
        """
        self.send_json_msg(json.dumps({str_key: str_value}))

In [279]:
class PosObj(Enum):
    """
    物体的位置
    """
    before_1st = 0  # 在1号对管之前
    at_1st = 1  # 在1号对管处
    between_1st_and_2nd = 2 # 在1,2号对管之间
    at_2nd = 3  # 在2号对管处
    between_2nd_and_3rd = 4 # 在2,3号对管之间
    at_3rd = 5  # 在3号对管处
    beyond_3rd = 6  # 在3号对管之后

In [280]:
class StateRod(Enum):
    """
    推杆状态
    """
    stop = 0    # 停止
    push = 1    # 推出
    pull = 2    # 拉回

In [281]:
class FS_IIOTA():
    """
    表征智能分拣系统行为的类
    """
    def __init__(self, ip_mqtt_broker, website_back_end):
        """
        parameters:ip_mqtt_broker:Mqtt broker的IP地址
                   website_back_end:用于识别的服务端(后端)网址
        """
        self.mqtt_clt = Mqtt_Clt(ip_mqtt_broker, 1883, "bb", "aa",
                                50)    # 构造Mqtt客户端
        self.website_back_end = website_back_end    # 获取用于识别的后端网址
        self.stat_stack_rod = StateRod.stop # 认为当前上货杆(1号推杆)停止
        self.stat_classify_rod = StateRod.stop  # 认为当前分类杆(2/3/4号推杆停止)
        self.mqtt_clt.control_device("conveyor", "run") # 控制传送带运行
        ####################################################
        time.sleep(3) 
        ####################################################
        self.timer_stack_rod = threading.Timer(1, self.handle_timer_statck_rod)  
        self.timer_classify_rod = None  # 初始化分类杆再次动作的定时器
        # 先发送消息将货推出再将上货杆设置为推出状态
        self.mqtt_clt.control_device("rod_control", "first_push")   # 控制上货杆推
        self.stat_stack_rod = StateRod.push # 将上货杆的状态设为推出
        self.timer_stack_rod.start()    # 启动上货杆再次动作的定时器
        # 定义3个分层队列保存每个对象
        self.q1 = deque()   
        self.q2 = deque()   
        self.q3 = deque()   

    def __del__(self):
        self.mqtt_clt.control_device("rod_control", "all_pull") # 控制所有推杆拉回
        time.sleep(3)
        self.mqtt_clt.control_device("conveyor", "stop")    # 控制传送带停止
        time.sleep(1)

    def handle_mqtt_json_msg(self, total_tolerance):
        """
        functions:以循环的方式对Mqtt客户端的有效消息进行处理,在发生异常的时候记录异常总次数;并触发相应的状态机
        """
        cnt_except = 0  # 定义发生异常的计数量(初始化为0)
        while cnt_except < total_tolerance: # 在发生异常的计数量小于容忍值时一直循环
            if self.mqtt_clt.msg != {}: # 在mqtt接收到的消息非空时
                # 这里并不是每0.3s就有图片发过来的 应该是有图片过的时候才发送消息 
                if "image" in self.mqtt_clt.msg:    # 如果接收到的是图像的话
                    img_data = base64.b64decode(self.mqtt_clt.msg["image"]) # 解码获得图像数据
                    try:
                        result = json.loads(requests.post(self.website_back_end, data=img_data).text)   # 将图像数据img_data给POST出去并获得相应的结果result
                        try:
                            assert len(result["results"]) == 1  # 确保识别结果result中的键"results"中的值只有1个
                            label = result["results"][0]["label"]   # 获得识别结果中的label
                            # print("lable=",label)
                            # 这是在图像采集区域，位置都是1号对管之前
                            if label == "ripe": 
                                self.q1.appendleft(2)
                            elif label == "half-ripe":  
                                self.q1.appendleft(3)
                            elif label == "raw":   
                                self.q1.appendleft(4)
                            else:
                                print("三分类以外的分支")
                        except: # 获得结果时发生异常,则cnt_except自加1
                            print("未获得有效结果!")
                            cnt_except += 1
                            print("cnt_except += 1")
                    except: # post时发生异常,则cnt_except自加1
                        print("无法post!")
                        cnt_except += 1
                        print("cnt_except += 1")
                elif "first_switch" in self.mqtt_clt.msg: # 如果接收到的是1号对管的消息 判断是进还是出 此时obj对象已被创建
                    if not self.mqtt_clt.msg["first_switch"]:  # 出1号对管
                        obj = self.q1.pop()
                        # 这里判断是否在2号分类杆这里分类-如果不分类则加入下一个队列
                        # 0.3
                        # threading.Timer(0.3, self.state_machine,(obj,)).start() if obj == 2 else self.q2.appendleft(obj) 
                        threading.Thread(target=self.state_machine,args=(obj,)).start() if obj == 2 else self.q2.appendleft(obj) 
                elif "second_switch" in self.mqtt_clt.msg:  # 如果接收到的是2号对管的消息
                    if not self.mqtt_clt.msg["second_switch"]:  # 进2号对管
                        obj = self.q2.pop()
                        # self.state_machine(obj) if obj == 3 else self.q3.appendleft(obj) 
                        # 0.4
                        # threading.Timer(0.4, self.state_machine,(obj,)).start() if obj == 3 else self.q3.appendleft(obj) 
                        threading.Thread(target=self.state_machine,args=(obj,)).start() if obj == 3 else self.q3.appendleft(obj) 
                elif "third_switch" in self.mqtt_clt.msg:   # 如果接收到的是3号对管的消息
                    if not self.mqtt_clt.msg["third_switch"]:   # 进3号对管
                        obj = self.q3.pop()
                        # 这里有一个坑！，如果每个分类杆单独行动 会出现ripe和raw杆同时推出的问题 同时发送消息导致消息发送失败(应该) 分类杆就收不回去
                        # 0.5
                        # threading.Timer(0.4, self.state_machine,(4,)).start()
                        threading.Thread(target=self.state_machine,args=(obj,)).start()
                        # self.state_machine(4)

                self.mqtt_clt.msg = {}  # 清空mqtt接收到的消息

    def handle_timer_statck_rod(self):
        """
        functions:对上货杆的状态进行分情况讨论:
        如果当前上货杆状态为推出,说明上货杆已经完全推出,需要令上货杆拉回;
        否则,如果上货杆状态为拉回,说明上货杆已经完全拉回,需要将上货杆的状态改为停止
        """
        if self.stat_stack_rod == StateRod.push:
            # print("上货杆已push,现在pull")
            # 发送将第一个杆(货杆)拉回的消息 然后设置为pull状态
            self.mqtt_clt.control_device("rod_control", "first_pull")
            self.stat_stack_rod = StateRod.pull
            ####################################################
            # 设置一个再次调用上货杆的定时器  
            # self.timer_stack_rod = threading.Timer(random.uniform(1,1.5), self.handle_timer_statck_rod) # 原3.5s
            self.timer_stack_rod = threading.Timer(1, self.handle_timer_statck_rod) # 原3.5s
            self.timer_stack_rod.start()
        elif self.stat_stack_rod == StateRod.pull:
            # print("上货杆已pull,现在push")
            self.stat_stack_rod = StateRod.stop
            ####################################################
            self.mqtt_clt.control_device("rod_control", "first_push")
            self.stat_stack_rod = StateRod.push
            self.timer_stack_rod = threading.Timer(0.5, self.handle_timer_statck_rod)
            self.timer_stack_rod.start()
    

    def handle_timer_classify_rod(self,act_num):
        """
        functions:对分类杆的状态进行分情况讨论:
        如果当前分类杆的状态为推出,说明分类杆已完全推出,需要让相应的分类杆拉回
        否则,如果当前分类杆的状态为拉回,说明分类杆已完全拉回,需要重置分类杆相关成员并令上货杆推出
        """
        # print(f"{act_num}号杆需要拉回")
        # if self.stat_classify_rod == StateRod.push:
        str_classify_rod_cmd = ""
        if act_num == 2:
            # str_classify_rod_cmd = "second"
            str_classify_rod_cmd = "second_pull"
            self.mqtt_clt.control_device("rod_control", str_classify_rod_cmd)
        elif act_num == 3:
            # str_classify_rod_cmd = "third"
            str_classify_rod_cmd = "third_pull"
            self.mqtt_clt.control_device("rod_control", str_classify_rod_cmd)
        elif act_num == 4:
            # str_classify_rod_cmd = "fourth"
            str_classify_rod_cmd = "fourth_pull"
            self.mqtt_clt.control_device("rod_control", str_classify_rod_cmd)
        # else:
        #     print("出现第四条路m,GGGG")
            # str_classify_rod_cmd += "_pull"
            # self.mqtt_clt.control_device("rod_control", str_classify_rod_cmd)
            # self.stat_classify_rod = StateRod.pull  # 分类杆状态变为拉回
            ####################################################
            # 启动分类杆动作相关定时器 
            # self.timer_classify_rod = await threading.Timer(0.35, self.handle_timer_classify_rod,(act_num,)) # 原3s
            # self.timer_classify_rod.start()
        #     self.handle_timer_classify_rod(act_num) 
        # elif self.stat_classify_rod == StateRod.pull:
        #     self.stat_classify_rod = StateRod.stop  # 分类杆状态为停止
        #     print(f"{act_num}号杆已停止")

           

    # 对状态机进行修改，只负责控制传来的act_num对应的分类杆
    def state_machine(self,act_num):
        """
        functions:以状态机的方法通过物体当前位置以及将要推出的分类杆的编号控制分类杆/上货杆推出,并对相应的状态成员变量进行重设
        """
        # print(f"{act_num}号杆需要推出")
        if act_num  == 2:  # 当需要2号推杆动作的时候
            # 推出2号推杆并启动分类杆相关定时器(为了完成后续的动作)
            self.mqtt_clt.control_device("rod_control", "second_push")
            # print("2号杆推出消息发出")
            self.stat_classify_rod = StateRod.push
            ####################################################
            # Timer时间后拉回分类杆
            self.timer_classify_rod = threading.Timer(0.3, self.handle_timer_classify_rod,(2,)).start()
            # self.timer_classify_rod.start()
            # threading.Thread(target=self.handle_timer_classify_rod,args=((2,))).start()
        elif act_num == 3:    # 当需要3号推杆动作的时候
            # 推出3号推杆并启动分类杆相关定时器(为了完成后续的动作)
            self.mqtt_clt.control_device("rod_control", "third_push")
            # print("3号杆推出消息发出")
            self.stat_classify_rod = StateRod.push
            ####################################################
            self.timer_classify_rod = threading.Timer(0.4, self.handle_timer_classify_rod,(3,)).start()
            # self.timer_classify_rod.start()
            # threading.Thread(target=self.handle_timer_classify_rod,args=((3,))).start()
        elif act_num == 4:    # 当需要4号推杆动作的时候
            # 推出4号推杆并启动分类杆相关定时器(为了完成后续的动作)
            self.mqtt_clt.control_device("rod_control", "fourth_push")    
            # print("4号杆推出消息发出")  
            self.stat_classify_rod = StateRod.push
            ####################################################
            self.timer_classify_rod = threading.Timer(0.5, self.handle_timer_classify_rod,(4,)).start()
            # self.timer_classify_rod.start()
            # threading.Thread(target=self.handle_timer_classify_rod,args=((4,))).start()
           

In [ ]:
if __name__ == "__main__":
    fs_iiota = FS_IIOTA("127.0.0.1", "http://127.0.0.1:8088/aisim_tf_pre")    # 构造类FS_IIOTA的对象
    fs_iiota.handle_mqtt_json_msg(5)    # 调用成员函数处理通过Mqtt发来的json消息

Exception in thread Thread-11364:
Traceback (most recent call last):
  File "c:\Users\JIA\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\JIA\AppData\Local\Programs\Python\Python310\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
TypeError: 'MQTTMessageInfo' object is not callable


{"rod_control": "first_push"}


Exception in thread Thread-11366:
Traceback (most recent call last):
  File "c:\Users\JIA\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\JIA\AppData\Local\Programs\Python\Python310\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
TypeError: 'MQTTMessageInfo' object is not callable


{"rod_control": "first_pull"}


Exception in thread Thread-11368:
Traceback (most recent call last):
  File "c:\Users\JIA\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\JIA\AppData\Local\Programs\Python\Python310\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
TypeError: 'MQTTMessageInfo' object is not callable


...

{"rod_control": "first_pull"}


Exception in thread Thread-11440:
Traceback (most recent call last):
  File "c:\Users\JIA\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\JIA\AppData\Local\Programs\Python\Python310\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
TypeError: 'MQTTMessageInfo' object is not callable


{"rod_control": "first_push"}


Exception in thread Thread-11442:
Traceback (most recent call last):
  File "c:\Users\JIA\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\JIA\AppData\Local\Programs\Python\Python310\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
TypeError: 'MQTTMessageInfo' object is not callable


{"rod_control": "first_pull"}
